In [1]:
import numpy as np
import pandas as pd

In [2]:
import pytorch_lightning as pl
import torch
from torch.utils.data import DataLoader
from urb3d.segmentation.dataset import PointCloudSegmentationDataset, PointSampler, ChunkedPointCloudDataset
from urb3d.segmentation.segmentor import PointNetSegmentor

from sklearn.metrics import precision_recall_fscore_support

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cpu')

In [3]:
model = '../models/weighted.ckpt'

In [4]:
weighted_model = '../models/pointnet_weighted/best_results/epoch=31-val_loss=1.99-train_loss=1.35.ckpt'
unweighted_model = '../models/new_deal/best_results/epoch=89-val_loss=1.81-train_loss=1.32.ckpt'
# block_6_model = '../models/longrun_pointnet_weighted_ins/best_results/epoch=1-val_loss=2.42-train_loss=2.51.ckpt'
block_7_model = '../models/pointnet_weighted_isns/best_results/epoch=113-val_loss=0.79-train_loss=1.18.ckpt'
block_6_model = ('../models/pointnet_weighted_isns_6/best_results/epoch=45-val_loss=1.22-train_loss=1.41.ckpt')
block_6_7_model = ('../models/pointnet_weighted_isns_6_7/best_results/epoch=64-val_loss=2.01-train_loss=1.34.ckpt')

In [5]:
model = PointNetSegmentor.load_from_checkpoint(model, strict=False)
dataset = ChunkedPointCloudDataset('../data/birmingham_blocks/test')
# dataset = PointCloudSegmentationDataset('../data/birmingham_blocks/birmingham_block_7_subsampled_test.ply')
data_loader = DataLoader(dataset)
trainer = pl.Trainer()
preds = trainer.predict(model, data_loader)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
C:\Users\48791\miniconda3\envs\zpi\Lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:424: The 'predict_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=7` in the `DataLoader` to improve performance.


Predicting: |          | 0/? [00:00<?, ?it/s]

In [14]:
# preds = preds[0][0].cpu().numpy()
# labels = data_loader.__iter__().__next__()[1][0].cpu().numpy()

IndexError: invalid index to scalar variable.

In [15]:
predictions = preds

In [6]:
labels = np.array([])
predictions = np.array([])

for idx, chunk in enumerate(data_loader):
    chunk_labels = chunk[1][0].cpu().numpy()
    labels = np.concatenate([labels, chunk_labels])
    chunk_preds = preds[idx][0].cpu().numpy()
    predictions = np.concatenate([predictions, chunk_preds])


In [20]:
metrics = precision_recall_fscore_support(labels, predictions)
for class_id in range(len(metrics[0])):
    for metric in range(4):
        print(round(metrics[metric][class_id],2), end='\t\t')
    print()
print(precision_recall_fscore_support(labels, predictions, average='weighted', zero_division=0))

0.26		0.92		0.41		42219		
0.0		0.0		0.0		14913		
0.83		0.93		0.87		75278		
0.0		0.0		0.0		7040		
0.0		0.0		0.0		25200		
0.51		0.06		0.11		59383		
0.0		0.0		0.0		6470		
0.0		0.0		0.0		8447		
(0.43365283395803866, 0.46968403431680267, 0.3749632518906431, None)


C:\Users\48791\miniconda3\envs\zpi\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [9]:
metrics = precision_recall_fscore_support(labels, predictions)
for class_id in range(len(metrics[0])):
    for metric in range(4):
        print(round(metrics[metric][class_id],2), end='\t\t')
    print()

print(precision_recall_fscore_support(labels, predictions, average='weighted'))

C:\Users\48791\miniconda3\envs\zpi\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


0.09		0.34		0.15		1800121		
0.13		0.01		0.02		2549658		
0.52		0.56		0.54		10126741		
0.01		0.04		0.02		244829		
0.0		0.0		0.0		173127		
0.15		0.04		0.07		2862766		
0.01		0.0		0.0		210120		
0.02		0.01		0.01		311217		
0.0		0.0		0.0		1229631		
0.0		0.0		0.0		998		
(0.3168028954279854, 0.3308348550079532, 0.30565188930756876, None)


C:\Users\48791\miniconda3\envs\zpi\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [ ]:
2

In [19]:
labels.shape

torch.Size([1, 238950])

In [11]:
preds[0][0].shape

torch.Size([238950])

In [9]:
for pred in preds:
    print(pred.shape)

torch.Size([1, 238950])


In [9]:
len(data_loader.dataset.pt.points)

238950

In [7]:
data_loader.dataset.point_sampler.subsample_size

In [15]:
print(data_loader.dataset.subsample_size)

238950


In [6]:
data_loader.__iter__().__next__()[0].shape

torch.Size([1, 3])

In [13]:
preds[0].shape

torch.Size([1, 1024])